# Homework 05
I. Take the airline opinion dataset (`airline_tweets.csv`) that was the subject of the Class 12  
II. Divide the dataset into Train/Test: 80/20 percent fix the randomness!  
III. Compare the different Methods:  

  1. TextBlob
  2. Microsoft Azure!!!
  3. a train your own model with Sklearn algorithm, compare on the same subset!
    * try different algorithms (https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html , play with parameters they offer) 
  3. * b try to train your model on completely new datasets LABELED (https://www.kaggle.com/datasets?search=sentiment&sort=votes&tags=13302-Classification&page=3)
    * maybe improve your cleaning algorithm

IV. Compare the results in the table  
V. Write conclusions, 
   * how woud you detect sentences showing: hatred/racism/sexism, in Twitter (for example)
   * which method would you choose to clasify opinions on Amazon products: positive/negative 

The Follow work is done by:

Joseph Nicolay Ruiz A., Physicist, University of Antioquia

Dic, 2022

## Loading and cleaning of data 
Let's load the data `airline_tweets.csv` treated during class 

In [321]:
import pandas as pd

data_source_url = "https://raw.githubusercontent.com/mhemmg/datasets/master/nlp/airline_tweets.csv"
airline_tweets = pd.read_csv(data_source_url)
airline_tweets.to_csv("airline_tweets.csv", index=False)
airline_tweets.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


Only it's necessary columns [`airline_sentiment`, `text`]

In [322]:
airline_tweets1 = airline_tweets[['airline_sentiment','text']].copy()
airline_tweets1.airline_sentiment.dropna(inplace=True)
airline_tweets1.drop_duplicates(inplace=True, ignore_index=True)
#airline_tweets1.to_csv('full.csv',index=False)
print(airline_tweets1.shape)
airline_tweets1.head()

(14452, 2)


,airline_sentiment,text
0,neutral,@VirginAmerica What @dhepburn said.
1,positive,@VirginAmerica plus you've added commercials t...
2,neutral,@VirginAmerica I didn't today... Must mean I n...
3,negative,@VirginAmerica it's really aggressive to blast...
4,negative,@VirginAmerica and it's a really big bad thing...


Now, we need to clean the `text`column, eliminating: user accounts, numeric and special character, stop words, etc.

In [330]:
import re
import spacy 
#from spacy.cli import download
#print(download('en_core_web_sm'))
from nlppreprocess import NLP
nlp_ = NLP()
nlp = spacy.load("en_core_web_sm")
stopwords = spacy.lang.en.stop_words.STOP_WORDS

In [357]:
def cleaner(feature):    

    # removing tweet
    processed_feature = re.sub(r'@\w+', ' ', str(feature))

    # removing retweet
    processed_feature = re.sub(r'rt @\w+:', ' ', processed_feature)

    # removing numeric
    processed_feature = re.sub("\d+", ' ', processed_feature)

    # Removing links
    processed_feature = re.sub(r'http\S+', ' ', processed_feature)

    # Remove all the special characters except (') 
    processed_feature = re.sub(r"[^\w']", ' ', processed_feature)

    # Remove single characters
    processed_feature = re.sub(r'^[a-zA-Z]\s+|\s+[a-zA-Z]\s+|\s+[a-zA-Z]$', ' ', processed_feature) 

    # Substituting multiple spaces with single space
    processed_feature = re.sub(r'\s+', ' ', processed_feature, flags=re.I)

    # Converting to Lowercase
    processed_feature = processed_feature.lower()

    # Removing StopWords
    #Note: It'snt conveniente remove stopwords as usual for sentiment analysis.
    #processed_feature = " ".join(token.lemma_ for token in nlp(processed_feature) if not token.is_stop)
    processed_feature = nlp_.process(processed_feature) #This method is better
    
    return processed_feature    
    

In [358]:
import numpy as np
collections = np.array_split(airline_tweets1.index.to_numpy(),10)

In [359]:
import multiprocessing as mp

def run_cleaning(collection, df=airline_tweets1):
    data = [cleaner(tweet) for tweet in df.text[collection]]
    return data

In [360]:
if __name__ == "__main__":
    pool = mp.Pool(processes= mp.cpu_count()-1)
    tasks = collections
    results = pool.map(run_cleaning, tasks)
    pool.close()
    pool.join()
    
airline_tweets1['text_processed'] = [tweet for result in results for tweet in result]

It yields the following result

In [361]:
airline_tweets1.head()

,airline_sentiment,text,text_processed
0,neutral,@VirginAmerica What @dhepburn said.,what said
1,positive,@VirginAmerica plus you've added commercials t...,plus you ve added commercials experience tacky
2,neutral,@VirginAmerica I didn't today... Must mean I n...,not today must mean need take another trip
3,negative,@VirginAmerica it's really aggressive to blast...,it s really aggressive blast obnoxious enterta...
4,negative,@VirginAmerica and it's a really big bad thing...,and it s really big bad thing about


Let's eliminate some irregularities ouputs such as 'empty' results after cleaning process.

In [362]:
mask  = airline_tweets1.text_processed == ' '
airline_tweets1 = airline_tweets1.drop(airline_tweets1[mask].index).reset_index(drop=True)

Define the features and labels for problem since the last above data frame columns 

In [363]:
features = airline_tweets1.iloc[:, 1].values
features_processed = airline_tweets1.iloc[:, 2].values
labels = airline_tweets1.iloc[:, 0].values

## Application of models 

Let's star applying `TexBlob`. The results are:

In [466]:
from textblob import TextBlob
from sklearn.metrics import accuracy_score
import numpy as np

In [365]:
def get_tweet_sentiment(tweet):    
    
    analysis = TextBlob(tweet)
    
    if analysis.sentiment.polarity > 0:
        return 'positive'
    elif analysis.sentiment.polarity == 0:
        return 'neutral'
    else:
        return 'negative' 
    
def Text_Blob(tweets):
    return list(map(get_tweet_sentiment, tweets))

In [366]:
TextBlob_Classifier = Text_Blob(features)
print(f'Non-processed: {accuracy_score(labels, TextBlob_Classifier):.2f}')
TextBlob_Classifier = Text_Blob(features_processed)
print(f'Processed: {accuracy_score(labels, TextBlob_Classifier):.2f}')

Non-processed: 0.46
Processed: 0.46


Besides, using `Microsoft Azure` (Cloud computing):


In [453]:
key = "c008ac7aa1244c1a9b469bd8282355af" 
endpoint = "https://jnra.cognitiveservices.azure.com/" 

from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

# Authenticate the client using your key and endpoint 
def authenticate_client():
    ta_credential = AzureKeyCredential(key)
    text_analytics_client = TextAnalyticsClient(
            endpoint=endpoint, 
            credential=ta_credential)
    return text_analytics_client

cliente = authenticate_client()

In [454]:
def sentiment_analysis_azure(documents):
    
    result = client.analyze_sentiment(list(documents), show_opinion_mining=False)
    doc_result = [doc.sentiment for doc in result if not doc.is_error]
    
    return doc_result

To avoid overpass the call volume quota for TextAnalytics F0 pricing tier, we're going to consider a sample smaller tha working-complete data set of tweets

In [455]:
import random
random.seed(21)
sample = np.sort(random.sample(range(len(labels)), 400))

labels_sample = labels[sample]
features_sample = features_processed[sample]

In [462]:
tasks = np.array_split(features_sample, len(features_sample)/10)
if __name__ == "__main__":
    pool = mp.Pool(processes= mp.cpu_count()-1)
    results = pool.map(sentiment_analysis_azure, list(tasks))
    pool.close()
    pool.join()
    
azure_Classifier =[sentiment for result in results for sentiment in result]

In [465]:
print(f'Processed: {accuracy_score(labels_sample, azure_Classifier):.2f}')

Processed: 0.60


Now, we can train our own model with Sklearn algorithm and compare.
Firstly, vectorize the tweets.

In [477]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(features_processed).toarray()

Splitting of train, test data set.

In [480]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=21)

In [483]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import LinearSVC

I'm going to implement with parameter set up by default. It's possible optimize using GridSearch from Sklearn 

`RadomForestClassifier`

In [482]:
RF_Classifier = RandomForestClassifier()
RF_Classifier.fit(X_train, y_train)
predictions = RF_Classifier.predict(X_test)
print(accuracy_score(y_test, predictions))

0.7540643375994466


`GradientBoostingClassifier`

In [ ]:
GB_Classifier = GradientBoostingClassifier()
GB_Classifier.fit(X_train, y_train)
predictions = GB_Classifier.predict(X_test)
print(accuracy_score(y_test, predictions))

`MLPClassifier`

In [ ]:
MLP_Classifier = MLPClassifier()
MLP_Classifier.fit(X_train, y_train)
predictions = MLP_Classifier.predict(X_test)
print(accuracy_score(y_test, predictions))

`AdaBoostClassifier`

In [ ]:
AB_classifier = AdaBoostClassifier()
AB_classifier.fit(X_train, y_train)
predictions = AB_classifier.predict(X_test)
print(accuracy_score(y_test, predictions))

`LinearSVC`

In [487]:
LSVC_Classifier = LinearSVC()
LSVC_Classifier.fit(X_train, y_train)
predictions = LSVC_Classifier.predict(X_test)
print(accuracy_score(y_test, predictions))

0.7838118298166724


### Conclusion
For implement pre-training model is fundamental how we clean (process) the text to be examined. Remove directly all stopWords may conclude wrong results for sentiment analysis. Besides, although we didn't adjust the best parameters for train models, we can conclude the best, as in time as in perfomance is the model `LinearSVC`, with an accuracy close to 80%. 

The results yielded by Microsoft Azure conclude additional information as subjetivity and weight for each type of possible sentiment. With this last information we can classify a tweet (or any other comment) as hatred,racism,sexism, etc. For example:
Let be ('negative'=x, 'positive'=y, 'neutral'=z), such a way there exist some unique relation among numerical weights (x,y,z) for each category (sexist,racist,...).

Another option, it consists on implement HugginFace (Yes, another pre-trained model).